In [1]:
from datasets import load_dataset
single_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling_singleturn")
multi_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling")
glaive_ds = load_dataset("NousResearch/hermes-function-calling-v1", "glaive_func_calling")

/Users/admin/fish/planck-agent/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import re
from pprint import pprint


def map_func(example):
    example["conversation"] = example["conversations"]
    del example["conversations"]
    messages = []
    if example["id"] == "e39781a4-29a3-4896-83d2-0ab5d264b6ed":
        example["conversation"] = messages
        example["functions"] = None
        return example
    try:
        for c in example["conversation"]:
            if c["from"] == "system":
                tools = re.search(r'<tools>\n(.*?)\n</tools>', c["value"], re.DOTALL).group(1)
                example["functions"] = json.dumps([t["function"] for t in eval(tools)], ensure_ascii=False)
            elif c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = json.dumps([eval(t) for t in tool_calls], ensure_ascii=False)
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = None
                messages.append({"role": "assistant", "content": content, "function_call": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                tool_rsps = json.dumps([eval(t) for t in tool_rsps], ensure_ascii=False)
                messages.append({"role": "function", "content": tool_rsps})
    except:
        messages = []
        example["functions"] = None
    example["conversation"] = messages
    return example


def glaive_map_func(example):
    example["conversation"] = example["conversations"]
    del example["conversations"]

    messages = []
    try:
        for c in example["conversation"]:
            if c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = json.dumps([eval(t) for t in tool_calls], ensure_ascii=False)
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = None
                messages.append({"role": "assistant", "content": content, "function_call": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                tool_rsps = json.dumps([{"name": json.loads(messages[-1]["function_call"])[0]["name"], "content": eval(t)} for t in tool_rsps], ensure_ascii=False)
                messages.append({"role": "function", "content": tool_rsps})
    except:
        messages = []
    if json.loads(example["tools"]):
        try:
            example["functions"] = json.dumps([t["function"] for t in json.loads(example["tools"])], ensure_ascii=False)
        except:
            messages = []
            example["functions"] = None
    else:
        example["functions"] = None
    example["conversation"] = messages
    example["subcategory"] = ""
    return example

In [3]:
ds1 = single_ds["train"].map(map_func).filter(lambda x: x["conversation"] and x["functions"])
ds2 = multi_ds["train"].map(map_func).filter(lambda x: x["conversation"] and x["functions"])
ds3 = glaive_ds["train"].map(glaive_map_func).filter(lambda x: x["conversation"] and x["functions"])
ds3 = ds3.remove_columns(['tools', 'source'])

from datasets import DatasetDict

all_ds = DatasetDict({
    "func_calling_singleturn": ds1,
    "func_calling": ds2,
    "glaive_func_calling": ds3
})
all_ds

Filter: 100%|██████████| 5209/5209 [00:00<00:00, 30510.74 examples/s]


DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1832
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1653
    })
    glaive_func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 4343
    })
})

In [4]:
import checker
import importlib
importlib.reload(checker)
ck = checker.Checker()

all_ds = all_ds.filter(lambda x: ck.check(x) is None, load_from_cache_file=False)
all_ds

Filter: 100%|██████████| 4343/4343 [00:03<00:00, 1381.00 examples/s]


DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1783
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1604
    })
    glaive_func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 4298
    })
})

In [6]:
import numpy as np
from collections import defaultdict
fc_num_stat = defaultdict(lambda: 0)

def filter_func(data):
    fcs = json.loads(data["functions"])
    if len(data["conversation"]) >= 10:
        if any(np.array([fc_num_stat[f["name"]] for f in fcs]) > 200):
            return False
        else:
            for f in fcs:
                fc_num_stat[f["name"]] += 1
            return True
    elif len(data["conversation"]) > 6:
        if any(np.array([fc_num_stat[f["name"]] for f in fcs]) > 150):
            return False
        else:
            for f in fcs:
                fc_num_stat[f["name"]] += 1
            return True
    else:
        if any(np.array([fc_num_stat[f["name"]] for f in fcs]) > 100):
            return False
        else:
            for f in fcs:
                if fc_num_stat[f["name"]] < 50:
                    for f in fcs:
                        fc_num_stat[f["name"]] += 1
                    return True
    return False
    
all_ds["glaive_func_calling"] = all_ds["glaive_func_calling"].filter(filter_func, load_from_cache_file=False)
all_ds

Filter: 100%|██████████| 4298/4298 [00:00<00:00, 18620.90 examples/s]


DatasetDict({
    func_calling_singleturn: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1783
    })
    func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 1604
    })
    glaive_func_calling: Dataset({
        features: ['id', 'category', 'subcategory', 'task', 'conversation', 'functions'],
        num_rows: 2810
    })
})

In [6]:
from collections import defaultdict
function_stat = defaultdict(lambda: 0)
for sample in all_ds['func_calling_singleturn']:
    if not sample["functions"]:
        function_stat[None] += 1
    else:
        functions = json.loads(sample["functions"]) 
        for f in functions:
            function_stat[f["name"]] += 1

sorted_stat = sorted(function_stat.items(), key=lambda x: x[1], reverse=True)
print(sorted_stat)
len(sorted_stat)

[('ExpertQAExtractor', 732), ('analyze_customer_feedback', 9), ('add_to_cart', 6), ('schedule_maintenance', 6), ('generate_inventory_report', 5), ('translate_text', 5), ('execute_trades', 5), ('place_market_order', 5), ('monitor_stock_levels', 5), ('update_inventory', 5), ('schedule_appointment', 5), ('analyze_network_traffic', 4), ('get_product_details', 4), ('transfer_funds', 4), ('analyze_sentiment', 4), ('identify_top_selling_products', 4), ('process_sale_transaction', 4), ('apply_discount', 4), ('extract_key_phrases', 4), ('reschedule_appointment', 3), ('cancel_order', 3), ('get_account_balance', 3), ('transcribe_audio', 3), ('synthesize_speech', 3), ('categorize_expenses', 3), ('generate_trading_signals', 3), ('predict_maintenance_needs', 3), ('generate_compliance_report', 3), ('update_inventory_stock', 3), ('process_payment', 3), ('segment_customers', 3), ('send_confirmation_email', 3), ('update_order_status', 3), ('generate_sales_report', 3), ('generate_transaction_report', 3),

2488

In [7]:
n = 0
for d in all_ds['func_calling_singleturn']:
    for c in d["conversation"]:
        if f := c["function_call"]:
            if len(json.loads(f)) > 1:
                n += 1
                break

n

584

In [8]:
all_ds.push_to_hub("hqfx/hermes_fc_cleaned")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/hqfx/hermes_fc_cleaned/commit/facb1242a96a200db525ea9921f671e51938ec54', commit_message='Upload dataset', commit_description='', oid='facb1242a96a200db525ea9921f671e51938ec54', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/hqfx/hermes_fc_cleaned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='hqfx/hermes_fc_cleaned'), pr_revision=None, pr_num=None)